In [2]:
%run ../yp_utils.py

# Initial setup

In [3]:
paper_pmid = 21252230
paper_name = 'martin_cunningham_2011' 

In [4]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [5]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [6]:
original_data = pd.read_excel('raw_data/1-s2.0-S0021925820538970-mmc1.xls', sheet_name='data')

In [7]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4955 x 16


In [8]:
original_data.head()

,ORF,gene,n(0),n(FK),n(TM),n(TMFK),n(AF),n(AFFK),MD-6,MD-2,flags,group,line,cluster,SL's,VPS
0,YOL143C,RIB4,0.3491,0.6636,1.0601,1.7492,0.9022,2.8714,827.112647,45.083915,NaN,A,1.0,few cells,0,NaN
1,YNR010W,CSE2,0.3036,0.7391,0.2786,1.7014,1.3013,2.2578,929.808483,59.825139,NaN,A,2.0,few cells,0,NaN
2,YHR120W,MSH1,0.4282,0.8490,0.4423,0.7674,1.2522,2.3904,1051.644722,30.112191,NaN,A,3.0,few cells,0,NaN
3,YPR072W,NOT5,0.5751,0.4358,0.5329,0.8912,0.2764,0.5106,1720.249369,36.935031,NaN,A,4.0,few cells,0,NaN
4,YOR008C-a,NaN,0.7507,0.3424,0.8165,0.5677,0.2083,0.5106,1967.032166,63.329453,NaN,A,5.0,few cells,0,NaN


In [9]:
original_data['orf'] = original_data['ORF'].astype(str)

In [10]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [11]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [12]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

            ORF gene    n(0)   n(FK)    n(TM)  n(TMFK)    n(AF)  n(AFFK)  \
index_input                                                                
776          wt  NaN  1.2337  2.2135   8.7616  50.9251  14.0312  44.6603   
4379         wt  NaN  1.2755  2.5045  11.0827  44.4959  20.0904  42.1672   
4527         wt  NaN  1.2327  1.9896   9.2019  45.9105  15.0678  40.4559   

                  MD-6       MD-2  flags group  line cluster  SL's  VPS orf  
index_input                                                                  
776          46.157065  11.959187    NaN     C   NaN     NaN     0  NaN  WT  
4379         17.330361  17.692191    NaN    wt   NaN     NaN     0  NaN  WT  
4527         20.125595   7.786368    NaN    wt   NaN     NaN     0  NaN  WT  


In [13]:
original_data = original_data.loc[t,:]

In [14]:
original_data.set_index('orf', inplace=True)

In [15]:
data_cols = ['n(0)', 'n(FK)', 'n(TM)', 'n(TMFK)', 'n(AF)', 'n(AFFK)']

In [16]:
original_data = original_data[data_cols].apply(pd.to_numeric, axis=1, errors='coerce')

In [17]:
original_data = original_data.groupby(original_data.index).mean()

In [18]:
original_data.shape

(4756, 6)

In [20]:
# Normalize by control (n(0))
original_data = original_data.div(original_data['n(0)'], axis=0)

In [22]:
original_data.drop(columns='n(0)', inplace=True)

# Prepare the final dataset

In [23]:
data = original_data.copy()

In [24]:
dataset_ids = [21844, 21843, 21841, 21842, 21840]
datasets = datasets.reindex(index=dataset_ids)

In [25]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [26]:
data.head()

dataset_id,21844,21843,21841,21842,21840
data_type,value,value,value,value,value
orf,,,,,
YAL002W,2.559678,4.218115,9.980415,7.404586,16.775724
YAL004W,1.189413,6.587335,39.399486,18.413111,41.103082
YAL005C,1.209389,5.009878,23.363619,14.758142,35.169291
YAL007C,0.953318,5.268621,35.972676,10.208635,32.839640
YAL008W,1.344249,7.160894,49.305206,13.253254,40.336587


## Subset to the genes currently in SGD

In [27]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [28]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,21844,21843,21841,21842,21840
,data_type,value,value,value,value,value
gene_id,orf,,,,,
2,YAL002W,2.559678,4.218115,9.980415,7.404586,16.775724
1863,YAL004W,1.189413,6.587335,39.399486,18.413111,41.103082
4,YAL005C,1.209389,5.009878,23.363619,14.758142,35.169291
5,YAL007C,0.953318,5.268621,35.972676,10.208635,32.839640
6,YAL008W,1.344249,7.160894,49.305206,13.253254,40.336587


# Normalize

In [29]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [30]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [31]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,21844,21843,21841,21842,21840,21844,21843,21841,21842,21840
,data_type,value,value,value,value,value,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,
2,YAL002W,2.559678,4.218115,9.980415,7.404586,16.775724,2.497680,-0.989706,-2.207680,-1.119656,-1.555388
1863,YAL004W,1.189413,6.587335,39.399486,18.413111,41.103082,-0.136606,0.346264,0.176755,1.736748,0.660980
4,YAL005C,1.209389,5.009878,23.363619,14.758142,35.169291,-0.098203,-0.543242,-1.122963,0.788386,0.120376
5,YAL007C,0.953318,5.268621,35.972676,10.208635,32.839640,-0.590490,-0.397340,-0.100990,-0.392083,-0.091869
6,YAL008W,1.344249,7.160894,49.305206,13.253254,40.336587,0.161060,0.669686,0.979620,0.397909,0.591147


# Print out

In [32]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [33]:
from IO.save_data_to_db3 import *

In [34]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/5 [00:00<?, ?it/s]

Deleting all datasets for PMID 21252230...
Inserting the new data...


100%|██████████| 5/5 [00:36<00:00,  7.26s/it]

Updating the data_modified_on field...
